# Cell correction

This notebook is for cells labels by deleting them, merge 2 neighboors cells or eventually extends them

# 0. Import 

In [1]:
from correct_cells import * 

In [2]:
import os
import copy
from pathlib import Path
import pandas as pd

import numpy as np
import napari
from napari.utils.notebook_display import nbscreenshot
from tqdm import tqdm
from rich.pretty import pprint

from dask.array.image import imread as imr


# 1. Load images and mask

In [3]:
images_path = Path("/media/raphael/data/works/20231220_C3_14_2h_ofON/bleach_correction/hist_splitt/C3.14_ON_01_w1SPI-488_s4/MIP")
labels_path = "/media/raphael/data/works/20231220_C3_14_2h_ofON/C3.14_ON_01_w1SPI-488_s4/full_movie/MIP/maskfolder/mask/" + "*.tif"

In [4]:
cellpose_path="/media/raphael/data/works/notebooks/notebook_scripts_perso/full_pipeline_perso/cellpose/"  + "*.tif"
cellpose=imr(cellpose_path)
cellpose = cellpose.compute()

In [5]:
labels = imr(labels_path)
labels = labels.compute()


In [6]:
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(images_path.glob("*.tif")), stack=True)
#viewer.open(sorted(labels_path.glob("*.tif")), stack=True)

[<Image layer 'C3.14_ON_01_w1SPI-488_s4_t001' at 0x7f634e3f0a90>]

In [7]:
images = viewer.layers[0].data
Label = viewer.add_labels(labels)
#CP = viewer.add_labels(cellpose)

In [26]:
cellIndexFolder = "/media/raphael/data/works/20231220_C3_14_2h_ofON/bleach_correction/hist_splitt/C3.14_ON_01_w1SPI-488_s4/MIP"
cellIndexName = "fichier"
cellIndexFile = cellIndexFolder + cellIndexName

# 2. Correct cells  

## 2.1 get Number and index of cells present on all frames 

In [22]:
percentage,cells_res = get_cells_numbers_on_all_frames(labels)

previous number of cells :  79
actual max number of cells :  75
number of cells present from first to last frame :  38
percentage of cells present from first to last frame :  50.66666666666667 %


## 2.1.2 save cell index  

In [23]:
cells_res

[0,
 9,
 10,
 12,
 13,
 18,
 20,
 21,
 22,
 24,
 25,
 26,
 28,
 29,
 31,
 32,
 33,
 34,
 36,
 37,
 40,
 41,
 42,
 43,
 45,
 46,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 62,
 64]

In [27]:
np.save(cellIndexFile,cells_res,allow_pickle=True, fix_imports=True)

## 2.2 Delete cells 

In dlt put a tuple corresponding to the labels of cells you want to delete and these cell will be successively deleted by "delete_cells"

In [14]:
# tuple corresponding to the numbers of cell which will be deleted 
dlt=[4,23] 
len_d = int(len(dlt))

In [15]:
for i in range(0,len_d):
    print("cell to be deleted ", dlt[i])
    labels = delete_cells(labels, dlt[i])
    labels = np.array(labels)
    viewer.layers[1].data = labels #refresh labels on napari

cell to be deleted  4
cell  4 has been deleted on all frames
cell to be deleted  23
cell  23 has been deleted on all frames


## 2.3 Merge neighboor cells 

The same principle goes for merging cell, but this time you have to put the cells you wants to merge by pair 
like this nbr = (a,b,x,y)


a and b will be merge together, and it will be the same for x and y 

In [ ]:
nbr=(51,53)
len_nbr = int(len(nbr))

In [ ]:
for i in range(0,len_nbr,2):
    print("parent cell ", nbr[i]," child cell ",nbr[i+1])
    labels = merge_neighboor_cells(nbr[i], nbr[i+1],labels)
    labels = np.array(labels)
    viewer.layers[1].data = labels #refresh labels on napari

## 2.4 Extend cells 

If it works this function will eventually serve to extend badly detected cell

In [ ]:
ext=[35]
len_ext=int(len(ext))
applyNUmber = 33
minLen=0

In [ ]:
for i in range(0,len_ext):
    labels = extends_cells(labels,cellpose[0],ext[i],applyNUmber,minLen)
    labels = np.array(labels)
    viewer.layers[1].data = labels #refresh labels on napari

In [ ]:
viewer.layers[1].data = labels

# 3 Save Corrected labels 

In [21]:
savepath = "/media/raphael/data/works/20231220_C3_14_2h_ofON/C3.14_ON_01_w1SPI-488_s4/full_movie/MIP/maskfolder/mask/"
save_zarr_images(labels,savepath,"C3.14_ON_01_w1SPI-488_s","4","mask")

Saving frame |████████████████████████████████████████| 120/120 [100%] in 2:33.6
